# 📦 Chapter 17: The Ultimate 3D Workflow Summary

We have covered processing, analysis, and deep learning. This notebook consolidates everything into a single modular pipeline.

**Pipeline Steps:**
1.  **Ingest**: Load PLY/LAS/XYZ.
2.  **Pre-process**: Clean noise, adaptive downsampling.
3.  **Analyze**: Extract geometric features.
4.  **Structure**: Voxelize or Octreetize.
5.  **Cluster**: DBSCAN/RANSAC.
6.  **Reconstruct**: Create a Mesh.

In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree

## 1. Helper Functions (The Toolkit)

We define our improved functions from pervious chapters.

In [ ]:
def remove_noise(pcd, nb_neighbors=20, std_ratio=2.0):
    print("Cleaning noise...")
    cl, ind = pcd.remove_statistical_outlier(nb_neighbors, std_ratio)
    return cl

def extract_features(pcd, radius=0.1):
    print("Extracting features...")
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius, max_nn=30))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius*5, max_nn=100))
    return fpfh

def meshing(pcd, depth=8):
    print("Reconstructing surface...")
    # Ensure normals
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(10)
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=depth)
    # Remove ghosts
    vertices_to_remove = densities < np.quantile(densities, 0.05)
    mesh.remove_vertices_by_mask(vertices_to_remove)
    return mesh

## 2. The Auto-Pilot Pipeline

In [ ]:
def run_pipeline(file_path):
    # 1. Load
    try:
        pcd = o3d.io.read_point_cloud(file_path)
        print(f"Loaded {len(pcd.points)} points.")
    except:
        print("File not found, skipping.")
        return
    
    # 2. Clean
    pcd_clean = remove_noise(pcd)
    
    # 3. Downsample (if too big)
    if len(pcd_clean.points) > 100000:
        print("Downsampling...")
        pcd_clean = pcd_clean.voxel_down_sample(voxel_size=0.05)
    
    # 4. Mesh
    mesh = meshing(pcd_clean)
    
    # 5. Visualize
    print("Done. Visualizing...")
    o3d.visualization.draw_geometries([mesh], window_name="Pipeline Result")
    
    return mesh

# Run on a sample
# run_pipeline("../DATA/sample.ply")